# An extremely ill-advised attempt at making a reinforcement learner for assignment 2.

In [1]:

from tensorflow import keras
from hanabi_learning_environment import rl_env
from hanabi_learning_environment.rl_env import Agent
import matplotlib.pyplot as plt


In [ ]:

import matplotlib.pyplot
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from datetime import datetime
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from typing import List, Tuple
import math

def calculate_reinforce_gradient1(trajectory, total_reward, baseline, action_choices):
    # This function is meant to calculate (dL/d Theta), where L=(\sum_t (log(P_t))(R-b).
    # You need to use the functions tf.math.log and tf.reduce_sum in here, plus the multiplication and subtraction operations.
    with tf.GradientTape() as tape:
        tape.watch(table_action_probabilities_before_softmax)
        trajectory_action_probabilities=get_action_probabilities_for_trajectory(trajectory) # this returns a tensor of shape [trajectory_length,4]
        chosen_probabilities=tf.gather(trajectory_action_probabilities, indices=action_choices, axis=1, batch_dims=1) # this returns a tensor of shape [trajectory_length]
        log_probabilities=tf.math.log(chosen_probabilities)
        L=tf.reduce_sum(log_probabilities)*(total_reward-baseline)
    assert len(L.shape)==0 # checking the original large array has gone through a reduce_sum
    grads = tape.gradient(L, table_action_probabilities_before_softmax) # This calculates the gradient required by REINFORCE
    # This function doesn't actually do the update.  It just calculates the gradient ascent direction, and returns it!
    return grads

def calculate_reinforce_gradient(episode_observations, rewards_minus_baseline, episode_actions, policy_network):
    # This function is meant to calculate (dL/d Theta), where L=(\sum_t (log(P_t))(R-b).

    # Train on episode
    batch_trajectory=np.stack(episode_observations)
    batch_action_choices=np.stack(episode_actions).astype(np.int32)

    # Check all input arrays are the correct shape...
    assert batch_trajectory.shape[0]==rewards_minus_baseline.shape[0]
    assert batch_trajectory.shape[0]==batch_action_choices.shape[0]
    assert len(batch_trajectory.shape)==2
    assert len(rewards_minus_baseline.shape)==1
    assert len(batch_action_choices.shape)==1

    # This is the REINFORCE gradient calculation
    with tf.GradientTape() as tape:
        # Note, don't need a tape.watch here because tensorflow by default always "watches" all Variable tensors, i.e. all of our neural network weights.
        trajectory_action_probabilities=policy_network(batch_trajectory)
        # Note that the next 2 lines could be repaced by a single call to tf.keras.losses.SparseCategoricalCrossentropy
        chosen_probabilities=tf.gather(trajectory_action_probabilities,indices=batch_action_choices,axis=1, batch_dims=1) # this returns a tensor of shape [trajectory_length]
        log_probabilities=tf.math.log(chosen_probabilities)
        logprobrewards=log_probabilities*rewards_minus_baseline # Instead of using R-baseline, we are using R_t-baseline here,
                                                                # i.e. where R_t is the reward to go from step t
        L=tf.reduce_sum(logprobrewards)
    assert len(L.shape)==0 # checking the original large array has gone through a reduce_sum
    grads = tape.gradient(L, policy_network.trainable_weights) # This calculates the gradient required by REINFORCE
    # This function doesn't actually do the update.  It just calculates the gradient ascent direction, and returns it!
    return grads


def calculate_accumulated_discounted_rewards(episode_rewards, discount_factor) -> np.ndarray:
    #discounted_episode_rewards= np.ones_like(episode_rewards)*sum(episode_rewards)

    discounts = list(discount_factor ** i for i in range(len(episode_rewards)))

    return np.array(
        list(
            sum(episode_rewards[i + j] * (discounts[j]) for j in range(0, len(episode_rewards) - i))
            for i in range(len(episode_rewards))
        )
    )
    # We need to return a numpy array of the same length and shape as the input array episode_rewards.


def run_stochastic_policy(policy_network, observation):
    # Reshape observation to (1,num_features)
    observation = observation[np.newaxis,:]
    # Run forward propagation to get softmax probabilities
    action_probabilities = policy_network(observation).numpy().reshape(-1)
    # Select action using a biased sample
    # this will return the index of the action we've sampled
    action = np.random.choice(range(len(action_probabilities)), p=action_probabilities)
    return action





In [ ]:

class MyAgent(Agent):

    def __init__(self, config, *args, **kwargs):
        self.config = config
        # Extract max info tokens or set default to 8.
        self.max_information_tokens: int = config.get('information_tokens', 8)
        pass

    def reset(self, config):
        pass

    def act(self, observation):
        if observation['current_player_offset'] != 0:
            return None # No action returned (because it's not our turn!)
